In [1]:
import pandas as pd
import numpy as np

In [ ]:
pd.MultiIndex()

In [10]:
import tensorflow as tf
import numpy as np

In [14]:
class Cash(tf.keras.Model):

    def __init__(self):
        super(Cash, self).__init__()

        self.dense = tf.keras.layers.Dense(5, activation='relu')
 
        self.b = tf.Variable(tf.zeros((1, 1), dtype=tf.float32), trainable=True)
        self.out = tf.keras.layers.Activation('softmax')

    def call(self, x):
        x = self.dense(x)
        bias = tf.tile(self.b, [tf.shape(x)[0], 1])
        x = tf.concat((bias, x), axis = -1)
        x = self.out(x)
        
        return x
    
    
model = Cash()

X = np.random.randn(100, 3)
with tf.GradientTape() as tape:
    y = model(X)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, model.trainable_variables)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

['Variable:0', 'cash_6/dense_6/kernel:0', 'cash_6/dense_6/bias:0']


In [16]:
class Cash(tf.keras.Model):

    def __init__(self, rows = 11, cols = 50, features = 3):
        super(Cash, self).__init__()
        
        input_shape = (rows, cols, features)
        inputs = tf.keras.Input(shape=input_shape)
        x = tf.keras.layers.Conv2D(
            filters = 2, 
            kernel_size = (1,3), 
            padding='valid', 
            activation='relu',
            name = 'conv1'
        )(inputs)
        x = tf.keras.layers.Conv2D(1, 
                                (1, x.shape[2]), 
                                activation="relu", 
                                name = 'conv2')(x)
        x = tf.squeeze(x)
        
        self.model = tf.keras.Model(inputs = inputs, outputs = x)
        
        self.b = tf.Variable(tf.zeros((1, 1), dtype=tf.float32), trainable=True)
        self.out = tf.keras.layers.Activation('softmax')

    def call(self, x):
        x = self.model(x)
        print(x.shape)
        bias = tf.tile(self.b, [tf.shape(x)[0], 1])
        x = tf.concat((bias, x), axis = -1)
        x = self.out(x)
        
        return x
    
    
model = Cash()

X = np.random.randn(100, 11, 50, 3)
with tf.GradientTape() as tape:
    y = model(X)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, model.trainable_variables)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(100, 11)
['conv1_1/kernel:0', 'conv1_1/bias:0', 'conv2_1/kernel:0', 'conv2_1/bias:0', 'Variable:0']


In [ ]:
batch = 100
dense = tf.keras.layers.Dense(5, activation='relu')
b = tf.Variable(tf.zeros((1,1), dtype=tf.float32), name='b', trainable=True)

x =  tf.random.normal((batch, 3))
with tf.GradientTape() as tape:
    y = dense(x)
    cash = tf.tile(b, [y.shape[0], 1])
    y = tf.concat((cash, y), axis = -1)
    y = tf.keras.layers.Activation('softmax')(y)

print([var.name for var in tape.watched_variables()])
grads = tape.gradient(y, [b, dense.trainable_variables])
grads

In [ ]:
def CNN(rows, cols, features, batch_size):
    input_shape = (rows, cols, features)
    X = keras.Input(shape= input_shape, batch_size=batch_size)
    w = keras.Input(shape = (rows, 1, 1), batch_size=batch_size)

    x = tf.keras.layers.Conv2D(
        filters = 2, 
        kernel_size = (1,3), 
        padding='valid', 
        activation='relu'
    )(X)
        
    x = keras.layers.Conv2D(20, 
                            (1, x.shape[2]), 
                            activation="relu", 
                            name = 'conv2')(x)  
    
    con = keras.layers.Concatenate(axis=3)([x, w])

    x = keras.layers.Conv2D(1, (1,1), name = 'votes')(con)
    x = tf.squeeze(x)
    
    b = tf.tile(b, [x.shape[0], 1])
    with_bias = keras.layers.Concatenate(axis=1)([b, x])
    #with_bias = CashBias()(x)

    outputs = keras.layers.Activation('softmax')(with_bias)
    return keras.Model(inputs = [X, w], outputs = outputs, name = "Policy")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
inputs = keras.Input(shape=(784,), name="digits")
x1 = layers.Dense(64, activation="relu")(inputs)
x2 = layers.Dense(64, activation="relu")(x1)
outputs = layers.Dense(10, name="predictions")(x2)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_train, (-1, 784))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
for (x, y) in train_dataset:

    with tf.GradientTape() as tape:

        # Run the forward pass of the layer.
        # The operations that the layer applies
        # to its inputs are going to be recorded
        # on the GradientTape.
        logits = model(x, training=True)  # Logits for this minibatch

        # Compute the loss value for this minibatch.
        loss_value = loss_fn(y, logits)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.
    grads = tape.gradient(loss_value, model.trainable_weights)

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    # optimizer.apply_gradients(zip(grads, model.trainable_weights))
    shapes = [grad.shape for grad in grads]
    print(shapes)
    print(model.trainable_variables)
    break

In [18]:
a = [np.random.rand(2,3) for i in range(10)]

In [26]:
np.array(a, dtype='float32').dtype

dtype('float32')